In [1]:
%%time

display("==== Starting 0-1 Image Acquisition ====")

import datetime as dt
import os
from datetime import datetime
import pandas
import pandas as pd
import praw
from praw.models import ListingGenerator
from tqdm import tqdm

from common.captioning.caption import BlipCaption
from common.schemas.pyarrow_schema import schema
from common.storage.azure_file_storage import AzureFileStorageAdapter
from common.functions.functions import Functions

tqdm.pandas(desc="Progress")

file_system = AzureFileStorageAdapter('data').get_file_storage()

functions: Functions = Functions()

CPU times: total: 3.31 s
Wall time: 3.61 s


In [2]:
%%time

# "memes","CityPorn", "EarthPorn", "spaceporn",	"itookapicture", "trippinthroughtime"
subs = [
	"SFWRedheads",
	"sfwpetite",
	"SFWNextDoorGirls",
	"fatsquirrelhate",
	"realasians",
	"KoreanHotties",
	"prettyasiangirls",
	"AsianOfficeLady",
	"mildlypenis",
	"AsianInvasion",
	"sexygirls",
	"PrettyGirls",
	"gentlemanboners",
	"hotofficegirls",
	"tightdresses",
	"DLAH",
	"bathandbodyworks",
	"AesPleasingAsianGirls"
]

CPU times: total: 0 ns
Wall time: 0 ns


In [3]:
%%time

sources = [
	{"name": "CityDiffusion", "data": ["CityPorn"]},
	{"name": "NatureDiffusion", "data": ["EarthPorn"]},
	{"name": "CosmicDiffusion", "data": ["spaceporn"]},
	{"name": "ITAPDiffusion", "data": ["itookapicture"]},
	{"name": "MemeDiffusion", "data": ["memes"]},
	{"name": "TTTDiffusion", "data": ["trippinthroughtime"]},
	{"name": "WallStreetDiffusion", "data": ["wallstreetbets"]},
	{"name": "SexyDiffusion", "data": ["selfies", "Amicute", "amihot", "AmIhotAF", "HotGirlNextDoor"]},
	{"name": "FatSquirrelDiffusion", "data": ["fatsquirrelhate"]},
	{"name": "CelebrityDiffusion", "data": ["celebrities"]},
	{"name": "OldLadyDiffusion", "data": ["oldladiesbakingpies"]},
	{"name": "SWFPetite", "data": ["sfwpetite"]},
	{"name": "SFWMilfs", "data": ["cougars_and_milfs_sfw"]},
	{"name": "RedHeadDiffusion", "data": ["SFWRedheads"]},
	{"name": "NextDoorGirlsDiffusion", "data": ["SFWNextDoorGirls"]},
	{"name": "SexyAsianDiffusion","data": ["realasians", "KoreanHotties", "prettyasiangirls", "AsianOfficeLady", "AsianInvasion","AesPleasingAsianGirls"]},
	{"name": "MildlyPenisDiffusion", "data": ["mildlypenis"]},
	{"name": "PrettyGirlDiffusion","data": ["sexygirls", "PrettyGirls", "gentlemanboners", "hotofficegirls", "tightdresses", "DLAH"]},
	{"name": "CandleDiffusion", "data": ["bathandbodyworks"]}
]
sources_df = pd.DataFrame.from_records(sources)

CPU times: total: 0 ns
Wall time: 0 ns


In [4]:
%%time

extant_data = pandas.read_parquet("data/processed_raw_data.parquet", engine='pyarrow', filesystem=file_system,
								  schema=schema)
extant_data.set_index('id', inplace=True)
display(extant_data)

,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,
100rn7k,AmIhotAF,veritynicole,"hey, hows your new year going (23F)",a woman in a white shirt and black pants is po...,4bd00c19fa0ff2ade855e6d364b0760b,/r/AmIhotAF/comments/100rn7k/hey_hows_your_new...,https://i.redd.it/n7r47s0gkh9a1.jpg,100rn7k.jpg,data/image/100rn7k.jpg,SexyDiffusion,True,True,False,[]
1013bdt,AmIhotAF,RaulDea9286,36F - ITALIAN,arafed image of a woman in a bikini top,7c0d158cba8654ef1c635cbc5471d597,/r/AmIhotAF/comments/1013bdt/36f_italian/,https://i.redd.it/bg0wwdlt5k9a1.jpg,1013bdt.jpg,data/image/1013bdt.jpg,SexyDiffusion,True,True,True,[]
105mekt,AmIhotAF,lindaniz,interesting in good forward relationship (f24),a close up of a woman with red hair and a whit...,ba4a0962cca2266a741e1e1700589c04,/r/AmIhotAF/comments/105mekt/interesting_in_go...,https://i.redd.it/4avjshsz8naa1.jpg,105mekt.jpg,data/image/105mekt.jpg,SexyDiffusion,True,True,True,[]
105qvgl,AmIhotAF,CaitVLove11,Laughing is my favorite 😆,a woman in a blue tank top and shorts is smili...,27bfe82c37314a0bcf02ab72eaf3a9e5,/r/AmIhotAF/comments/105qvgl/laughing_is_my_fa...,https://i.redd.it/2pulzr0lxmaa1.jpg,105qvgl.jpg,data/image/105qvgl.jpg,SexyDiffusion,True,True,True,[]
105rpcj,AmIhotAF,Flashy-Desk1858,[f22] What do you think when you see me?,a woman in a blue bikini top and a blue bra top,329eb42b8267fa1cc2980da8e48bcef1,/r/AmIhotAF/comments/105rpcj/f22_what_do_you_t...,https://i.redd.it/rz68pf934naa1.jpg,105rpcj.jpg,data/image/105rpcj.jpg,SexyDiffusion,True,True,True,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131dso7,DLAH,Wallydinger123,Happy B Day,there are three women posing for a picture in ...,ebb4b4a88dc79e2f2f57b10f32bf03c0,/r/DLAH/comments/131dso7/happy_b_day/,https://i.redd.it/36gurz533jwa1.jpg,131dso7.jpg,data/image/131dso7.jpg,PrettyGirlDiffusion,True,True,False,[]
131eznp,DLAH,Long_dong111,Perfect,araffe woman in a white dress sitting on a pur...,5c944c6d62b1ae182c5a6a40c64195f4,/r/DLAH/comments/131eznp/perfect/,https://i.redd.it/dnbbx4zucjwa1.jpg,131eznp.jpg,data/image/131eznp.jpg,PrettyGirlDiffusion,True,True,True,[]
131g3am,DLAH,ThrownAwayMiles,DLAH left or right,two women in short dresses posing for a pictur...,ab1f8fc33c85f4cfce12c810cf22229e,/r/DLAH/comments/131g3am/dlah_left_or_right/,https://i.redd.it/zpp70oor3lwa1.jpg,131g3am.jpg,data/image/131g3am.jpg,PrettyGirlDiffusion,True,True,True,[]


CPU times: total: 422 ms
Wall time: 986 ms


In [5]:
%%time

curated_data = pandas.read_parquet("data/parquet/back.parquet", engine='pyarrow', filesystem=file_system, schema=schema)
curated_data.set_index('id', inplace=True)

display("==== CURATED DATA ====")
display(curated_data.shape)
display(curated_data)

'==== CURATED DATA ===='

(39719, 14)

,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,
100rn7k,AmIhotAF,veritynicole,"hey, hows your new year going (23F)",a woman in a white shirt and black pants is po...,4bd00c19fa0ff2ade855e6d364b0760b,/r/AmIhotAF/comments/100rn7k/hey_hows_your_new...,https://i.redd.it/n7r47s0gkh9a1.jpg,100rn7k.jpg,data/image/100rn7k.jpg,SexyDiffusion,True,True,False,[]
1013bdt,AmIhotAF,RaulDea9286,36F - ITALIAN,arafed image of a woman in a bikini top,7c0d158cba8654ef1c635cbc5471d597,/r/AmIhotAF/comments/1013bdt/36f_italian/,https://i.redd.it/bg0wwdlt5k9a1.jpg,1013bdt.jpg,data/image/1013bdt.jpg,SexyDiffusion,True,True,True,[]
105mekt,AmIhotAF,lindaniz,interesting in good forward relationship (f24),a close up of a woman with red hair and a whit...,ba4a0962cca2266a741e1e1700589c04,/r/AmIhotAF/comments/105mekt/interesting_in_go...,https://i.redd.it/4avjshsz8naa1.jpg,105mekt.jpg,data/image/105mekt.jpg,SexyDiffusion,True,True,True,[]
105qvgl,AmIhotAF,CaitVLove11,Laughing is my favorite 😆,a woman in a blue tank top and shorts is smili...,27bfe82c37314a0bcf02ab72eaf3a9e5,/r/AmIhotAF/comments/105qvgl/laughing_is_my_fa...,https://i.redd.it/2pulzr0lxmaa1.jpg,105qvgl.jpg,data/image/105qvgl.jpg,SexyDiffusion,True,True,True,[]
105rpcj,AmIhotAF,Flashy-Desk1858,[f22] What do you think when you see me?,a woman in a blue bikini top and a blue bra top,329eb42b8267fa1cc2980da8e48bcef1,/r/AmIhotAF/comments/105rpcj/f22_what_do_you_t...,https://i.redd.it/rz68pf934naa1.jpg,105rpcj.jpg,data/image/105rpcj.jpg,SexyDiffusion,True,True,True,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13no1on,DLAH,Majestic_Painter8660,"Tight dress, legs and heels",there is a woman sitting on a chair in a room,dbab04d6a37e17e6cc6df9cfc16ed51e,/r/DLAH/comments/13no1on/tight_dress_legs_and_...,https://i.redd.it/0pq456mci51b1.jpg,13no1on.jpg,data/image/13no1on.jpg,PrettyGirlDiffusion,True,False,False,[]
13nhylw,bathandbodyworks,BathbeautyXO,(Late) February/valentines empties! 💕❤️💖,there are many different types of skin care pr...,7d79ab2ad5d7fe73454db01b4351388c,/r/bathandbodyworks/comments/13nhylw/late_febr...,https://i.redd.it/t2yvxblvd51b1.jpg,13nhylw.jpg,data/image/13nhylw.jpg,CandleDiffusion,True,False,False,[]
13nlz14,bathandbodyworks,Fresh-Blueberry-milk,figured this would be appreciated 😏 also what ...,someone is holding a bottle of lotion in their...,8b7dfc23aeb352024f0a696ab49351ae,/r/bathandbodyworks/comments/13nlz14/figured_t...,https://i.redd.it/or6fujdmf61b1.jpg,13nlz14.jpg,data/image/13nlz14.jpg,CandleDiffusion,True,False,False,[]


CPU times: total: 250 ms
Wall time: 523 ms


In [6]:
%%time

extant_data.update(curated_data)
updated_extant_data = extant_data.reset_index()

display("==== UPDATED EXTANT DATA ====")
display(updated_extant_data.shape)
display(updated_extant_data)
updated_extant_data.to_parquet("data/processed_raw_data.parquet", engine='pyarrow', filesystem=file_system, schema=schema)

'==== UPDATED EXTANT DATA ===='

(41505, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
0,100rn7k,AmIhotAF,veritynicole,"hey, hows your new year going (23F)",a woman in a white shirt and black pants is po...,4bd00c19fa0ff2ade855e6d364b0760b,/r/AmIhotAF/comments/100rn7k/hey_hows_your_new...,https://i.redd.it/n7r47s0gkh9a1.jpg,100rn7k.jpg,data/image/100rn7k.jpg,SexyDiffusion,True,True,False,[]
1,1013bdt,AmIhotAF,RaulDea9286,36F - ITALIAN,arafed image of a woman in a bikini top,7c0d158cba8654ef1c635cbc5471d597,/r/AmIhotAF/comments/1013bdt/36f_italian/,https://i.redd.it/bg0wwdlt5k9a1.jpg,1013bdt.jpg,data/image/1013bdt.jpg,SexyDiffusion,True,True,True,[]
2,105mekt,AmIhotAF,lindaniz,interesting in good forward relationship (f24),a close up of a woman with red hair and a whit...,ba4a0962cca2266a741e1e1700589c04,/r/AmIhotAF/comments/105mekt/interesting_in_go...,https://i.redd.it/4avjshsz8naa1.jpg,105mekt.jpg,data/image/105mekt.jpg,SexyDiffusion,True,True,True,[]
3,105qvgl,AmIhotAF,CaitVLove11,Laughing is my favorite 😆,a woman in a blue tank top and shorts is smili...,27bfe82c37314a0bcf02ab72eaf3a9e5,/r/AmIhotAF/comments/105qvgl/laughing_is_my_fa...,https://i.redd.it/2pulzr0lxmaa1.jpg,105qvgl.jpg,data/image/105qvgl.jpg,SexyDiffusion,True,True,True,[]
4,105rpcj,AmIhotAF,Flashy-Desk1858,[f22] What do you think when you see me?,a woman in a blue bikini top and a blue bra top,329eb42b8267fa1cc2980da8e48bcef1,/r/AmIhotAF/comments/105rpcj/f22_what_do_you_t...,https://i.redd.it/rz68pf934naa1.jpg,105rpcj.jpg,data/image/105rpcj.jpg,SexyDiffusion,True,True,True,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41500,131dso7,DLAH,Wallydinger123,Happy B Day,there are three women posing for a picture in ...,ebb4b4a88dc79e2f2f57b10f32bf03c0,/r/DLAH/comments/131dso7/happy_b_day/,https://i.redd.it/36gurz533jwa1.jpg,131dso7.jpg,data/image/131dso7.jpg,PrettyGirlDiffusion,True,True,False,[]
41501,131eznp,DLAH,Long_dong111,Perfect,araffe woman in a white dress sitting on a pur...,5c944c6d62b1ae182c5a6a40c64195f4,/r/DLAH/comments/131eznp/perfect/,https://i.redd.it/dnbbx4zucjwa1.jpg,131eznp.jpg,data/image/131eznp.jpg,PrettyGirlDiffusion,True,True,True,[]
41502,131g3am,DLAH,ThrownAwayMiles,DLAH left or right,two women in short dresses posing for a pictur...,ab1f8fc33c85f4cfce12c810cf22229e,/r/DLAH/comments/131g3am/dlah_left_or_right/,https://i.redd.it/zpp70oor3lwa1.jpg,131g3am.jpg,data/image/131g3am.jpg,PrettyGirlDiffusion,True,True,True,[]
41503,131jvcx,DLAH,robok212,Good morning 😍,araffe in a pink dress posing on a white wall,01f5058855526ef25888cff88065db4c,/r/DLAH/comments/131jvcx/good_morning/,https://i.redd.it/ozmbfhjv3mwa1.jpg,131jvcx.jpg,data/image/131jvcx.jpg,PrettyGirlDiffusion,True,True,False,[]


CPU times: total: 328 ms
Wall time: 3.01 s


In [7]:
%%time

reddit: praw.Reddit = praw.Reddit(client_id='5hVavL0PIRyM_1JSvqT6UQ', client_secret='BjD2kS3WNLnJc59RKY-JJUuc_Z9-JA', user_agent='script:%(bot_name)s:v%(bot_version)s (by /u/%(bot_author)s)')

CPU times: total: 31.2 ms
Wall time: 78 ms


In [8]:
for sub in tqdm(subs, total=len(subs), desc="Creating Temp Dir For Subs..."):
	temp_dir_path_ = f"temp/{sub}"
	if not os.path.exists(temp_dir_path_):
		os.makedirs(temp_dir_path_)
	os.makedirs(temp_dir_path_, exist_ok=True)

Creating Temp Dir For Subs...: 100%|██████████| 18/18 [00:00<00:00, 8959.00it/s]


In [9]:
with open('log.txt', 'a') as f:
	for time_filter in ['week', 'day']:
		s = tqdm(subs, desc=f"{time_filter}", total=len(subs))
		for sub in s:
		# for sub in tqdm(subs, desc=f"{time_filter}", total=len(subs)):
			temp_dir_path = f"temp/{sub}"
			try:
				subreddit_stream: ListingGenerator = reddit.subreddit(display_name=sub).top(limit=100, time_filter=time_filter)
				subreddit_stream = list(subreddit_stream)
				p = tqdm(subreddit_stream, total=len(subreddit_stream), desc=f"Posts - {sub} - {time_filter}")
				for submission in p:
					if submission is None:
						continue
					else:
						if submission.id in extant_data.index.values:
							continue
					try:
						author_name = 'Unknown'
						subreddit_name = sub
						try:
							author_name = submission.author.name
						except Exception as e:
							author_name = 'Unknown'
							pass
						p = {
							'id': submission.id,
							'subreddit': subreddit_name,
							'author': author_name,
							'title': submission.title,
							'caption': '',
							'hash': '',
							'permalink': submission.permalink,
							'original_url': submission.url,
							'image_name': '',
							'path': '',
							'thumbnail_path': '',
							'exists': False,
							'curated': False,
							'Tags': ['']
						}
						pd.DataFrame([p]).to_parquet(f"{temp_dir_path}/{submission.id}.parquet")
					except Exception as e:
						log = functions.write_log_message(submission.id, sub, "Error Writing Post", e)
						f.write(log)
						continue
			except Exception as e:
				log = functions.write_log_message(submission.id, sub, f"Error Getting Posts For SubReddit", e)
				f.write(log)
				continue

day: 100%|██████████| 18/18 [03:09<00:00, 10.52s/it]


In [10]:
%%time

data = []
for sub in tqdm(subs, desc="Reading Temp Dir For Subs..."):
	df = pandas.read_parquet(f"temp/{sub}", schema=schema, engine='pyarrow')
	records = df.to_dict(orient='records')
	data.extend(records)

temp_data = pandas.DataFrame(data)

for i, r in temp_data.iterrows():
	temp_data.loc[i, 'image_name'] = r.id + ".jpg"
	temp_data.loc[i, 'path'] = ""
	temp_data.loc[i, 'hash'] = ""
	temp_data.loc[i, 'caption'] = ""
	temp_data.loc[i, 'model'] = ""
	temp_data.loc[i, 'exists'] = False
	temp_data.loc[i, 'curated'] = False
	temp_data.loc[i, 'accept'] = False
	temp_data.loc[i, 'tags'] = ['']

temp_data.set_index('id', inplace=True, drop=False)

display("==== TEMP DATA ====")
display(temp_data.shape)
display(temp_data)

Reading Temp Dir For Subs...: 100%|██████████| 18/18 [00:00<00:00, 20.86it/s]


'==== TEMP DATA ===='

(1992, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
13h67sb,13h67sb,SFWRedheads,Dasharkiss_real,Love life and life will love you too.,,,/r/SFWRedheads/comments/13h67sb/love_life_and_...,https://i.imgur.com/NBCDz3f.jpg,13h67sb.jpg,,,False,False,False,[]
13h93zj,13h93zj,SFWRedheads,thatgirlmarie97,Who wants to play some games?,,,/r/SFWRedheads/comments/13h93zj/who_wants_to_p...,https://www.reddit.com/gallery/13h93zj,13h93zj.jpg,,,False,False,False,[]
13h9lq5,13h9lq5,SFWRedheads,snildeep,Hey! My eyes are up here! 😂💕,,,/r/SFWRedheads/comments/13h9lq5/hey_my_eyes_ar...,https://i.redd.it/schip1qaotza1.jpg,13h9lq5.jpg,,,False,False,False,[]
13hc2uz,13hc2uz,SFWRedheads,nurselacey,How do you like my haircut?,,,/r/SFWRedheads/comments/13hc2uz/how_do_you_lik...,https://i.redd.it/0qgvz4uy9uza1.jpg,13hc2uz.jpg,,,False,False,False,[]
13hcvxr,13hcvxr,SFWRedheads,Puzzled_MJ,Happy Sunday. Where would u take me today?,,,/r/SFWRedheads/comments/13hcvxr/happy_sunday_w...,https://i.redd.it/h6bwkyw8guza1.jpg,13hcvxr.jpg,,,False,False,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13mh4fw,13mh4fw,AesPleasingAsianGirls,Korean_Celebrity_Fap,Arin Compilation,,,/r/AesPleasingAsianGirls/comments/13mh4fw/arin...,https://i.imgur.com/bukbo8v.gifv,13mh4fw.jpg,,,False,False,False,[]
13n9xdx,13n9xdx,AesPleasingAsianGirls,yummybanchan,Song Yu Qi,,,/r/AesPleasingAsianGirls/comments/13n9xdx/song...,https://i.redd.it/xl0v8fgbk31b1.jpg,13n9xdx.jpg,,,False,False,False,[]
13nm990,13nm990,AesPleasingAsianGirls,Defiant-Appearance64,Somi compilation,,,/r/AesPleasingAsianGirls/comments/13nm990/somi...,https://i.imgur.com/ISJoDGW.gifv,13nm990.jpg,,,False,False,False,[]


CPU times: total: 5.95 s
Wall time: 2.75 s


In [11]:
%%time

filtered = temp_data[~temp_data['id'].isin(extant_data.index.values) & temp_data['original_url'].str.endswith('.jpg') & (~temp_data['id'].isin(curated_data.index.values))]

filtered.dropna(axis=1, how='all')

filtered.reset_index(drop=True, inplace=True)

filtered.set_index('id', inplace=True, drop=False)

display("==== FILTERED DATA ====")

display(filtered.shape)

display(filtered)

'==== FILTERED DATA ===='

(84, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
13hgme0,13hgme0,SFWRedheads,Redhotwife88,I Love wearing my Leather Pants and Heels! [F],,,/r/SFWRedheads/comments/13hgme0/i_love_wearing...,https://i.redgifs.com/i/frayedmadeupsparrow.jpg,13hgme0.jpg,,,False,False,False,[]
13ibnsi,13ibnsi,SFWRedheads,Dasharkiss_real,I can live without money but I can't live with...,,,/r/SFWRedheads/comments/13ibnsi/i_can_live_wit...,https://i.redgifs.com/i/imaginaryrawleveret.jpg,13ibnsi.jpg,,,False,False,False,[]
13imeet,13imeet,SFWRedheads,imjenrose,Adorable girl to make your day better,,,/r/SFWRedheads/comments/13imeet/adorable_girl_...,https://i.redgifs.com/i/dangerousfilthyhornedv...,13imeet.jpg,,,False,False,False,[]
13jm7mh,13jm7mh,SFWRedheads,Redhotwife88,Do I look Cute for my Anniversary Dinner? [F],,,/r/SFWRedheads/comments/13jm7mh/do_i_look_cute...,https://i.redgifs.com/i/tensedarkmagentabedlin...,13jm7mh.jpg,,,False,False,False,[]
13nr68v,13nr68v,SFWRedheads,ComprehensiveCap1691,I hope i can make your day better with a littl...,,,/r/SFWRedheads/comments/13nr68v/i_hope_i_can_m...,https://i.redd.it/52i233v4u71b1.jpg,13nr68v.jpg,,,False,False,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13nwjqp,13nwjqp,DLAH,spicy_kingWest44,Gotta get them angles,,,/r/DLAH/comments/13nwjqp/gotta_get_them_angles/,https://i.redd.it/7xdtvt84l81b1.jpg,13nwjqp.jpg,,,False,False,False,[]
13nx3q5,13nx3q5,DLAH,robok212,I know there is not much dress ... but there i...,,,/r/DLAH/comments/13nx3q5/i_know_there_is_not_m...,https://i.redd.it/1jdres3dp81b1.jpg,13nx3q5.jpg,,,False,False,False,[]
13nt4ei,13nt4ei,bathandbodyworks,LadyMisfits,Nice surprise from Micheal’s,,,/r/bathandbodyworks/comments/13nt4ei/nice_surp...,https://i.redd.it/mse5auo4681b1.jpg,13nt4ei.jpg,,,False,False,False,[]


CPU times: total: 15.6 ms
Wall time: 21 ms


In [12]:
%%time

filtered['model'] = filtered.progress_apply(lambda x: functions.add_source(x, sources), axis=1)

display("== Filtered Data With Model ==")
display(filtered.shape)
display(filtered)

Progress: 100%|██████████| 84/84 [00:00<00:00, 84327.80it/s]
<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


'== Filtered Data With Model =='

(84, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
13hgme0,13hgme0,SFWRedheads,Redhotwife88,I Love wearing my Leather Pants and Heels! [F],,,/r/SFWRedheads/comments/13hgme0/i_love_wearing...,https://i.redgifs.com/i/frayedmadeupsparrow.jpg,13hgme0.jpg,,RedHeadDiffusion,False,False,False,[]
13ibnsi,13ibnsi,SFWRedheads,Dasharkiss_real,I can live without money but I can't live with...,,,/r/SFWRedheads/comments/13ibnsi/i_can_live_wit...,https://i.redgifs.com/i/imaginaryrawleveret.jpg,13ibnsi.jpg,,RedHeadDiffusion,False,False,False,[]
13imeet,13imeet,SFWRedheads,imjenrose,Adorable girl to make your day better,,,/r/SFWRedheads/comments/13imeet/adorable_girl_...,https://i.redgifs.com/i/dangerousfilthyhornedv...,13imeet.jpg,,RedHeadDiffusion,False,False,False,[]
13jm7mh,13jm7mh,SFWRedheads,Redhotwife88,Do I look Cute for my Anniversary Dinner? [F],,,/r/SFWRedheads/comments/13jm7mh/do_i_look_cute...,https://i.redgifs.com/i/tensedarkmagentabedlin...,13jm7mh.jpg,,RedHeadDiffusion,False,False,False,[]
13nr68v,13nr68v,SFWRedheads,ComprehensiveCap1691,I hope i can make your day better with a littl...,,,/r/SFWRedheads/comments/13nr68v/i_hope_i_can_m...,https://i.redd.it/52i233v4u71b1.jpg,13nr68v.jpg,,RedHeadDiffusion,False,False,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13nwjqp,13nwjqp,DLAH,spicy_kingWest44,Gotta get them angles,,,/r/DLAH/comments/13nwjqp/gotta_get_them_angles/,https://i.redd.it/7xdtvt84l81b1.jpg,13nwjqp.jpg,,PrettyGirlDiffusion,False,False,False,[]
13nx3q5,13nx3q5,DLAH,robok212,I know there is not much dress ... but there i...,,,/r/DLAH/comments/13nx3q5/i_know_there_is_not_m...,https://i.redd.it/1jdres3dp81b1.jpg,13nx3q5.jpg,,PrettyGirlDiffusion,False,False,False,[]
13nt4ei,13nt4ei,bathandbodyworks,LadyMisfits,Nice surprise from Micheal’s,,,/r/bathandbodyworks/comments/13nt4ei/nice_surp...,https://i.redd.it/mse5auo4681b1.jpg,13nt4ei.jpg,,CandleDiffusion,False,False,False,[]


CPU times: total: 15.6 ms
Wall time: 18 ms


In [13]:
%%time

file_list = file_system.ls("data/image")

filtered['path'] = filtered.progress_apply(lambda x: functions.fetch_image(x, file_list, file_system), axis=1)

display("== Filtered Data With Path ==")
display(filtered.shape)
display(filtered)

Progress: 100%|██████████| 84/84 [00:40<00:00,  2.09it/s]
<timed exec>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


'== Filtered Data With Path =='

(84, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
13hgme0,13hgme0,SFWRedheads,Redhotwife88,I Love wearing my Leather Pants and Heels! [F],,,/r/SFWRedheads/comments/13hgme0/i_love_wearing...,https://i.redgifs.com/i/frayedmadeupsparrow.jpg,13hgme0.jpg,,RedHeadDiffusion,False,False,False,[]
13ibnsi,13ibnsi,SFWRedheads,Dasharkiss_real,I can live without money but I can't live with...,,,/r/SFWRedheads/comments/13ibnsi/i_can_live_wit...,https://i.redgifs.com/i/imaginaryrawleveret.jpg,13ibnsi.jpg,,RedHeadDiffusion,False,False,False,[]
13imeet,13imeet,SFWRedheads,imjenrose,Adorable girl to make your day better,,,/r/SFWRedheads/comments/13imeet/adorable_girl_...,https://i.redgifs.com/i/dangerousfilthyhornedv...,13imeet.jpg,,RedHeadDiffusion,False,False,False,[]
13jm7mh,13jm7mh,SFWRedheads,Redhotwife88,Do I look Cute for my Anniversary Dinner? [F],,,/r/SFWRedheads/comments/13jm7mh/do_i_look_cute...,https://i.redgifs.com/i/tensedarkmagentabedlin...,13jm7mh.jpg,,RedHeadDiffusion,False,False,False,[]
13nr68v,13nr68v,SFWRedheads,ComprehensiveCap1691,I hope i can make your day better with a littl...,,,/r/SFWRedheads/comments/13nr68v/i_hope_i_can_m...,https://i.redd.it/52i233v4u71b1.jpg,13nr68v.jpg,data/image/13nr68v.jpg,RedHeadDiffusion,False,False,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13nwjqp,13nwjqp,DLAH,spicy_kingWest44,Gotta get them angles,,,/r/DLAH/comments/13nwjqp/gotta_get_them_angles/,https://i.redd.it/7xdtvt84l81b1.jpg,13nwjqp.jpg,data/image/13nwjqp.jpg,PrettyGirlDiffusion,False,False,False,[]
13nx3q5,13nx3q5,DLAH,robok212,I know there is not much dress ... but there i...,,,/r/DLAH/comments/13nx3q5/i_know_there_is_not_m...,https://i.redd.it/1jdres3dp81b1.jpg,13nx3q5.jpg,data/image/13nx3q5.jpg,PrettyGirlDiffusion,False,False,False,[]
13nt4ei,13nt4ei,bathandbodyworks,LadyMisfits,Nice surprise from Micheal’s,,,/r/bathandbodyworks/comments/13nt4ei/nice_surp...,https://i.redd.it/mse5auo4681b1.jpg,13nt4ei.jpg,data/image/13nt4ei.jpg,CandleDiffusion,False,False,False,[]


CPU times: total: 24 s
Wall time: 1min 18s


In [14]:
%%time

filtered['exists'] = filtered.progress_apply(lambda x: functions.set_exists(x), axis=1)

display("== Filtered Data With Exists ==")
display(filtered.shape)
display(filtered)

Progress: 100%|██████████| 84/84 [00:00<00:00, 16789.21it/s]
<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


'== Filtered Data With Exists =='

(84, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
13hgme0,13hgme0,SFWRedheads,Redhotwife88,I Love wearing my Leather Pants and Heels! [F],,,/r/SFWRedheads/comments/13hgme0/i_love_wearing...,https://i.redgifs.com/i/frayedmadeupsparrow.jpg,13hgme0.jpg,,RedHeadDiffusion,False,False,False,[]
13ibnsi,13ibnsi,SFWRedheads,Dasharkiss_real,I can live without money but I can't live with...,,,/r/SFWRedheads/comments/13ibnsi/i_can_live_wit...,https://i.redgifs.com/i/imaginaryrawleveret.jpg,13ibnsi.jpg,,RedHeadDiffusion,False,False,False,[]
13imeet,13imeet,SFWRedheads,imjenrose,Adorable girl to make your day better,,,/r/SFWRedheads/comments/13imeet/adorable_girl_...,https://i.redgifs.com/i/dangerousfilthyhornedv...,13imeet.jpg,,RedHeadDiffusion,False,False,False,[]
13jm7mh,13jm7mh,SFWRedheads,Redhotwife88,Do I look Cute for my Anniversary Dinner? [F],,,/r/SFWRedheads/comments/13jm7mh/do_i_look_cute...,https://i.redgifs.com/i/tensedarkmagentabedlin...,13jm7mh.jpg,,RedHeadDiffusion,False,False,False,[]
13nr68v,13nr68v,SFWRedheads,ComprehensiveCap1691,I hope i can make your day better with a littl...,,,/r/SFWRedheads/comments/13nr68v/i_hope_i_can_m...,https://i.redd.it/52i233v4u71b1.jpg,13nr68v.jpg,data/image/13nr68v.jpg,RedHeadDiffusion,True,False,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13nwjqp,13nwjqp,DLAH,spicy_kingWest44,Gotta get them angles,,,/r/DLAH/comments/13nwjqp/gotta_get_them_angles/,https://i.redd.it/7xdtvt84l81b1.jpg,13nwjqp.jpg,data/image/13nwjqp.jpg,PrettyGirlDiffusion,True,False,False,[]
13nx3q5,13nx3q5,DLAH,robok212,I know there is not much dress ... but there i...,,,/r/DLAH/comments/13nx3q5/i_know_there_is_not_m...,https://i.redd.it/1jdres3dp81b1.jpg,13nx3q5.jpg,data/image/13nx3q5.jpg,PrettyGirlDiffusion,True,False,False,[]
13nt4ei,13nt4ei,bathandbodyworks,LadyMisfits,Nice surprise from Micheal’s,,,/r/bathandbodyworks/comments/13nt4ei/nice_surp...,https://i.redd.it/mse5auo4681b1.jpg,13nt4ei.jpg,data/image/13nt4ei.jpg,CandleDiffusion,True,False,False,[]


CPU times: total: 15.6 ms
Wall time: 26 ms


In [15]:
%%time

filtered['hash'] = filtered.progress_apply(lambda x: functions.set_hash(x), axis=1)

display("== Filtered Data With Hash ==")
display(filtered.shape)
display(filtered)

Progress: 100%|██████████| 84/84 [00:00<00:00, 1023.75it/s]
<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


'== Filtered Data With Hash =='

(84, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
13hgme0,13hgme0,SFWRedheads,Redhotwife88,I Love wearing my Leather Pants and Heels! [F],,,/r/SFWRedheads/comments/13hgme0/i_love_wearing...,https://i.redgifs.com/i/frayedmadeupsparrow.jpg,13hgme0.jpg,,RedHeadDiffusion,False,False,False,[]
13ibnsi,13ibnsi,SFWRedheads,Dasharkiss_real,I can live without money but I can't live with...,,,/r/SFWRedheads/comments/13ibnsi/i_can_live_wit...,https://i.redgifs.com/i/imaginaryrawleveret.jpg,13ibnsi.jpg,,RedHeadDiffusion,False,False,False,[]
13imeet,13imeet,SFWRedheads,imjenrose,Adorable girl to make your day better,,,/r/SFWRedheads/comments/13imeet/adorable_girl_...,https://i.redgifs.com/i/dangerousfilthyhornedv...,13imeet.jpg,,RedHeadDiffusion,False,False,False,[]
13jm7mh,13jm7mh,SFWRedheads,Redhotwife88,Do I look Cute for my Anniversary Dinner? [F],,,/r/SFWRedheads/comments/13jm7mh/do_i_look_cute...,https://i.redgifs.com/i/tensedarkmagentabedlin...,13jm7mh.jpg,,RedHeadDiffusion,False,False,False,[]
13nr68v,13nr68v,SFWRedheads,ComprehensiveCap1691,I hope i can make your day better with a littl...,,9c76d407de60c8b396ffccd749b7d563,/r/SFWRedheads/comments/13nr68v/i_hope_i_can_m...,https://i.redd.it/52i233v4u71b1.jpg,13nr68v.jpg,data/image/13nr68v.jpg,RedHeadDiffusion,True,False,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13nwjqp,13nwjqp,DLAH,spicy_kingWest44,Gotta get them angles,,26a955b5d75d32d39ac452a9a934d15d,/r/DLAH/comments/13nwjqp/gotta_get_them_angles/,https://i.redd.it/7xdtvt84l81b1.jpg,13nwjqp.jpg,data/image/13nwjqp.jpg,PrettyGirlDiffusion,True,False,False,[]
13nx3q5,13nx3q5,DLAH,robok212,I know there is not much dress ... but there i...,,5fd63ab2fb5a39229ba0eba3a2775f8a,/r/DLAH/comments/13nx3q5/i_know_there_is_not_m...,https://i.redd.it/1jdres3dp81b1.jpg,13nx3q5.jpg,data/image/13nx3q5.jpg,PrettyGirlDiffusion,True,False,False,[]
13nt4ei,13nt4ei,bathandbodyworks,LadyMisfits,Nice surprise from Micheal’s,,362b6a403ebe68409fdefbcf7729ffbc,/r/bathandbodyworks/comments/13nt4ei/nice_surp...,https://i.redd.it/mse5auo4681b1.jpg,13nt4ei.jpg,data/image/13nt4ei.jpg,CandleDiffusion,True,False,False,[]


CPU times: total: 109 ms
Wall time: 103 ms


In [27]:
ts = datetime.today().timestamp()

exists = filtered.loc[filtered['exists'] == True]
exists.dropna(axis=1, how='all')
exists.reset_index(drop=True, inplace=True)

new_records_to_process = exists.copy()
del exists

new_records_to_process.reset_index(drop=True, inplace=True)
display(new_records_to_process)

out_name = f"data/temp/caption/{ts}_new_records_to_process.parquet"
new_records_to_process.to_parquet(out_name, engine='pyarrow', filesystem=file_system, schema=schema)
del new_records_to_process

new_df = pandas.read_parquet(out_name, engine='pyarrow', filesystem=file_system, schema=schema)

display("== New Records To Process ==")
display(new_df.shape)
display(new_df)
del new_df

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
0,13nr68v,SFWRedheads,ComprehensiveCap1691,I hope i can make your day better with a littl...,,9c76d407de60c8b396ffccd749b7d563,/r/SFWRedheads/comments/13nr68v/i_hope_i_can_m...,https://i.redd.it/52i233v4u71b1.jpg,13nr68v.jpg,data/image/13nr68v.jpg,RedHeadDiffusion,True,False,False,[]
1,13nrcar,SFWRedheads,AnnSweetBaby,"At first I wanted a pink skirt, but I decided ...",,62e7826405bdc556cb923a98498c9872,/r/SFWRedheads/comments/13nrcar/at_first_i_wan...,https://i.redd.it/9k0md38kv71b1.jpg,13nrcar.jpg,data/image/13nrcar.jpg,RedHeadDiffusion,True,False,False,[]
2,13nrkgm,SFWRedheads,Junior-Midnight-8241,Hey…you look like you need a hug 🫂,,80fe1c72b6bf976448ffb9161a5245f3,/r/SFWRedheads/comments/13nrkgm/heyyou_look_li...,https://i.redd.it/r0z64kzhx71b1.jpg,13nrkgm.jpg,data/image/13nrkgm.jpg,RedHeadDiffusion,True,False,False,[]
3,13nrutc,SFWRedheads,Dasharkiss_real,"today the weather is bad, we stay at home and ...",,f5da1c7c918ff25c06b53bc0ecf2d071,/r/SFWRedheads/comments/13nrutc/today_the_weat...,https://i.redd.it/yu2u0kvdi61b1.jpg,13nrutc.jpg,data/image/13nrutc.jpg,RedHeadDiffusion,True,False,False,[]
4,13ns48w,SFWRedheads,JenniferCoope,Hey! My eyes are up here! 😂💕,,80c2a694f5492f4885de0cd9553f02dd,/r/SFWRedheads/comments/13ns48w/hey_my_eyes_ar...,https://i.imgur.com/72fXRTa.jpg,13ns48w.jpg,data/image/13ns48w.jpg,RedHeadDiffusion,True,False,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,13nwjqp,DLAH,spicy_kingWest44,Gotta get them angles,,26a955b5d75d32d39ac452a9a934d15d,/r/DLAH/comments/13nwjqp/gotta_get_them_angles/,https://i.redd.it/7xdtvt84l81b1.jpg,13nwjqp.jpg,data/image/13nwjqp.jpg,PrettyGirlDiffusion,True,False,False,[]
68,13nx3q5,DLAH,robok212,I know there is not much dress ... but there i...,,5fd63ab2fb5a39229ba0eba3a2775f8a,/r/DLAH/comments/13nx3q5/i_know_there_is_not_m...,https://i.redd.it/1jdres3dp81b1.jpg,13nx3q5.jpg,data/image/13nx3q5.jpg,PrettyGirlDiffusion,True,False,False,[]
69,13nt4ei,bathandbodyworks,LadyMisfits,Nice surprise from Micheal’s,,362b6a403ebe68409fdefbcf7729ffbc,/r/bathandbodyworks/comments/13nt4ei/nice_surp...,https://i.redd.it/mse5auo4681b1.jpg,13nt4ei.jpg,data/image/13nt4ei.jpg,CandleDiffusion,True,False,False,[]
70,13nwdel,bathandbodyworks,lifeinsidethepage,Laundry Scoop 😉,,fdbf23831cd5776d1b93d902d626d4ec,/r/bathandbodyworks/comments/13nwdel/laundry_s...,https://i.redd.it/zr5pm949271b1.jpg,13nwdel.jpg,data/image/13nwdel.jpg,CandleDiffusion,True,False,False,[]


'== New Records To Process =='

(72, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
0,13nr68v,SFWRedheads,ComprehensiveCap1691,I hope i can make your day better with a littl...,,9c76d407de60c8b396ffccd749b7d563,/r/SFWRedheads/comments/13nr68v/i_hope_i_can_m...,https://i.redd.it/52i233v4u71b1.jpg,13nr68v.jpg,data/image/13nr68v.jpg,RedHeadDiffusion,True,False,False,[]
1,13nrcar,SFWRedheads,AnnSweetBaby,"At first I wanted a pink skirt, but I decided ...",,62e7826405bdc556cb923a98498c9872,/r/SFWRedheads/comments/13nrcar/at_first_i_wan...,https://i.redd.it/9k0md38kv71b1.jpg,13nrcar.jpg,data/image/13nrcar.jpg,RedHeadDiffusion,True,False,False,[]
2,13nrkgm,SFWRedheads,Junior-Midnight-8241,Hey…you look like you need a hug 🫂,,80fe1c72b6bf976448ffb9161a5245f3,/r/SFWRedheads/comments/13nrkgm/heyyou_look_li...,https://i.redd.it/r0z64kzhx71b1.jpg,13nrkgm.jpg,data/image/13nrkgm.jpg,RedHeadDiffusion,True,False,False,[]
3,13nrutc,SFWRedheads,Dasharkiss_real,"today the weather is bad, we stay at home and ...",,f5da1c7c918ff25c06b53bc0ecf2d071,/r/SFWRedheads/comments/13nrutc/today_the_weat...,https://i.redd.it/yu2u0kvdi61b1.jpg,13nrutc.jpg,data/image/13nrutc.jpg,RedHeadDiffusion,True,False,False,[]
4,13ns48w,SFWRedheads,JenniferCoope,Hey! My eyes are up here! 😂💕,,80c2a694f5492f4885de0cd9553f02dd,/r/SFWRedheads/comments/13ns48w/hey_my_eyes_ar...,https://i.imgur.com/72fXRTa.jpg,13ns48w.jpg,data/image/13ns48w.jpg,RedHeadDiffusion,True,False,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,13nwjqp,DLAH,spicy_kingWest44,Gotta get them angles,,26a955b5d75d32d39ac452a9a934d15d,/r/DLAH/comments/13nwjqp/gotta_get_them_angles/,https://i.redd.it/7xdtvt84l81b1.jpg,13nwjqp.jpg,data/image/13nwjqp.jpg,PrettyGirlDiffusion,True,False,False,[]
68,13nx3q5,DLAH,robok212,I know there is not much dress ... but there i...,,5fd63ab2fb5a39229ba0eba3a2775f8a,/r/DLAH/comments/13nx3q5/i_know_there_is_not_m...,https://i.redd.it/1jdres3dp81b1.jpg,13nx3q5.jpg,data/image/13nx3q5.jpg,PrettyGirlDiffusion,True,False,False,[]
69,13nt4ei,bathandbodyworks,LadyMisfits,Nice surprise from Micheal’s,,362b6a403ebe68409fdefbcf7729ffbc,/r/bathandbodyworks/comments/13nt4ei/nice_surp...,https://i.redd.it/mse5auo4681b1.jpg,13nt4ei.jpg,data/image/13nt4ei.jpg,CandleDiffusion,True,False,False,[]
70,13nwdel,bathandbodyworks,lifeinsidethepage,Laundry Scoop 😉,,fdbf23831cd5776d1b93d902d626d4ec,/r/bathandbodyworks/comments/13nwdel/laundry_s...,https://i.redd.it/zr5pm949271b1.jpg,13nwdel.jpg,data/image/13nwdel.jpg,CandleDiffusion,True,False,False,[]


In [28]:
display(f"Total Number Of New Images - {new_df.shape[0]}")
display("0-1 Image Acquisition Complete - Shutting Down")

'Total Number Of New Images - 72'

'0-1 Image Acquisition Complete - Shutting Down'

In [29]:
!jupyter notebook stop

^C


Error processing line 1 of D:\code\repos\simple-collection\venv\lib\site-packages\vision-1.0.0-py3.9-nspkg.pth:

  Traceback (most recent call last):
    File "C:\Users\AJ Stangl\AppData\Local\Programs\Python\Python39\lib\site.py", line 169, in addpackage
      exec(line)
    File "<string>", line 1, in <module>
    File "<frozen importlib._bootstrap>", line 562, in module_from_spec
  AttributeError: 'NoneType' object has no attribute 'loader'

Remainder of file ignored
Error processing line 1 of D:\code\repos\simple-collection\venv\lib\site-packages\vision-1.0.0-py3.9-nspkg.pth:

  Traceback (most recent call last):
    File "C:\Users\AJ Stangl\AppData\Local\Programs\Python\Python39\lib\site.py", line 169, in addpackage
      exec(line)
    File "<string>", line 1, in <module>
    File "<frozen importlib._bootstrap>", line 562, in module_from_spec
  AttributeError: 'NoneType' object has no attribute 'loader'

Remainder of file ignored
Error processing line 1 of D:\code\repos\simple-co